## 하이퍼파라미터(Hyperparameter) 튜닝으로 성능 올리기


1. **데이터셋을 불러옵니다.**

In [1]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


### 신경망에 교차 검증(Cross-Validation) 적용해보기

2. **필요한 라이브러리를 import 합니다.**

In [2]:
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import tensorflow as tf
import os

3. **`KFold`를 통해 학습 데이터셋을 몇 개(k)로 나눌지를 결정합니다.**

아래에서는 많이 사용되는 k인 5로 설정해보도록 하겠습니다.

In [3]:
kf = KFold(n_splits = 5)
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True) 

x_train.shape

(404, 13)

In [4]:
y_train[:5]

array([15.2, 42.3, 50. , 21.1, 17.7])

In [5]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0]), y_train):
    training_data = x_train.iloc[train_index]
    validation_data = x_train.iloc[val_index]
    training_y = y_train.iloc[train_index]
    validation_y = y_train.iloc[val_index]

In [6]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [7]:
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))


model.compile(loss='mean_squared_logarithmic_error',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
model.fit(training_data, training_y, epochs=2)

Epoch 1/2
11/11 [==============================] - 5s 4ms/step - loss: 9.5446 - accuracy: 0.0000e+00
Epoch 2/2
11/11 [==============================] - 0s 3ms/step - loss: 9.5446 - accuracy: 0.0000e+00


이제는 **교차 검증(Cross-Validation)을 적용할 차례**입니다.<br/>
다시 학습 데이터셋(**`x_train, y_train`**)을 k개 의 set으로 나누어주겠습니다. 

In [9]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0]),y_train):
    training_data = x_train.iloc[train_index, :]
    training_data_label = y_train.iloc[train_index]
    validation_data = x_train.iloc[val_index, :]
    validation_data_label = y_train.iloc[val_index]

다시 모델을 학습시켜줍니다.

In [10]:
model.fit(training_data, training_data_label,
			epochs=10,
            batch_size=64,
			validation_data=(validation_data, validation_data_label),
          )

Epoch 1/10
6/6 [==============================] - 0s 51ms/step - loss: 9.5446 - accuracy: 0.0000e+00 - val_loss: 9.9995 - val_accuracy: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 0s 16ms/step - loss: 9.5446 - accuracy: 0.0000e+00 - val_loss: 9.9995 - val_accuracy: 0.0000e+00
Epoch 3/10
6/6 [==============================] - 0s 8ms/step - loss: 9.5446 - accuracy: 0.0000e+00 - val_loss: 9.9995 - val_accuracy: 0.0000e+00
Epoch 4/10
6/6 [==============================] - 0s 9ms/step - loss: 9.5446 - accuracy: 0.0000e+00 - val_loss: 9.9995 - val_accuracy: 0.0000e+00
Epoch 5/10
6/6 [==============================] - 0s 11ms/step - loss: 9.5446 - accuracy: 0.0000e+00 - val_loss: 9.9995 - val_accuracy: 0.0000e+00
Epoch 6/10
6/6 [==============================] - 0s 8ms/step - loss: 9.5446 - accuracy: 0.0000e+00 - val_loss: 9.9995 - val_accuracy: 0.0000e+00
Epoch 7/10
6/6 [==============================] - 0s 10ms/step - loss: 9.5446 - accuracy: 0.0000e+00 - val_loss: 9.9995 -

데이터가 잘 나누어져 들어갔는지 확인해봅니다. 

In [11]:
print(training_data[:2])
print(training_data.shape)

        0     1     2    3      4      5     6       7    8      9     10  \
0  1.23247   0.0  8.14  0.0  0.538  6.142  91.7  3.9769  4.0  307.0  21.0   
1  0.02177  82.5  2.03  0.0  0.415  7.610  15.7  6.2700  2.0  348.0  14.7   

       11     12  
0  396.90  18.72  
1  395.38   3.11  
(324, 13)


In [12]:
training_data_label[:2]

,0
0,15.2
1,42.3


In [13]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(training_data, training_data_label,
          epochs=10,
          batch_size=32,
          )

Epoch 1/10
11/11 [==============================] - 1s 3ms/step - loss: 783.5084
Epoch 2/10
11/11 [==============================] - 0s 3ms/step - loss: 256.3284
Epoch 3/10
11/11 [==============================] - 0s 3ms/step - loss: 143.3844
Epoch 4/10
11/11 [==============================] - 0s 3ms/step - loss: 93.2796
Epoch 5/10
11/11 [==============================] - 0s 3ms/step - loss: 75.6038
Epoch 6/10
11/11 [==============================] - 0s 3ms/step - loss: 63.2057
Epoch 7/10
11/11 [==============================] - 0s 3ms/step - loss: 62.4458
Epoch 8/10
11/11 [==============================] - 0s 3ms/step - loss: 61.9120
Epoch 9/10
11/11 [==============================] - 0s 3ms/step - loss: 61.8049
Epoch 10/10
11/11 [==============================] - 0s 3ms/step - loss: 58.9959


In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(training_data, training_data_label,
          epochs=10,
          batch_size=32,
          )

Epoch 1/10
11/11 [==============================] - 1s 3ms/step - loss: -320.9759
Epoch 2/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153
Epoch 3/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4154
Epoch 4/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153
Epoch 5/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153
Epoch 6/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4154
Epoch 7/10
11/11 [==============================] - 0s 4ms/step - loss: -321.4153
Epoch 8/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153
Epoch 9/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153
Epoch 10/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153


In [15]:
results = model.evaluate(x_test, y_test, batch_size=32)
print("test loss, test mse:", results)

4/4 [==============================] - 0s 6ms/step - loss: -336.6793
test loss, test mse: -336.67926025390625


이제 한 번에 테스트를 수행해보겠습니다. 

In [16]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0])):
    training_data = x_train.iloc[train_index, :]
    training_data_label = y_train.iloc[train_index]
    validation_data = x_train.iloc[val_index, :]
    validation_data_label = y_train.iloc[val_index]

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(x_train, y_train,
              epochs=10,
              batch_size=32,
              validation_data = (validation_data, validation_data_label),
              )
    
    results = model.evaluate(x_test, y_test, batch_size=32)
    print("test loss, test mse:", results)

Epoch 1/10
13/13 [==============================] - 1s 14ms/step - loss: 2909.4329 - val_loss: 1284.2828
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 478.9255 - val_loss: 484.2084
Epoch 3/10
13/13 [==============================] - 0s 6ms/step - loss: 188.1732 - val_loss: 129.4544
Epoch 4/10
13/13 [==============================] - 0s 4ms/step - loss: 91.0652 - val_loss: 87.1426
Epoch 5/10
13/13 [==============================] - 0s 6ms/step - loss: 65.9513 - val_loss: 52.5152
Epoch 6/10
13/13 [==============================] - 0s 4ms/step - loss: 62.8512 - val_loss: 52.4922
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 60.7840 - val_loss: 56.5525
Epoch 8/10
13/13 [==============================] - 0s 5ms/step - loss: 61.0647 - val_loss: 49.8549
Epoch 9/10
13/13 [==============================] - 0s 5ms/step - loss: 58.9470 - val_loss: 50.5471
Epoch 10/10
4/4 [==============================] - 0s 5ms/step - loss: 60.8472
test loss, t

교차 검증을 통해서 모델을 돌릴 수 있는 것까지 확인해보았습니다.

## 신경망에서의 하이퍼 파라미터 튜닝

교차 검증 방법을 익혔으니 이제 본격적으로 하이퍼파라미터 튜닝을 시도해보겠습니다.<br/>
머신러닝(Section 2)에서 공부하셨던 것처럼 흔히 사용되는 하이퍼파라미터 튜닝 방법에는 아래와 같은 것들이 있습니다.

교차 검증 방법을 익혔으니 이제 본격적으로 하이퍼파라미터 튜닝을 시도해보겠습니다.<br/>
머신러닝(Section 2)에서 공부하셨던 것처럼 흔히 사용되는 하이퍼파라미터 튜닝 방법에는 아래와 같은 것들이 있습니다.

1. **"Babysitting"(육아) 혹은 "Grad Student Descent"(대학원생 갈아넣기)**

    다윈의 진화론을 아시나요? 진화론에서는 '자연 선택'이란 단어가 진화를 주도했다고 말하곤 합니다.<br/>
하지만 하이퍼 파라미터 선택은 자연이 해주지 않습니다. 그렇다면 우리가 직접 하는 수 밖에 없겠죠?<br/>
이전 프로젝트나 이번 스프린트에서 모델 성능을 높이기 위해 여러 숫자를 직접 넣어보며 하이퍼 파라미터를 수없이 조정했다면,<br/>
첫 번째 방법을 수행했다고 말할 수 있겠습니다.

    100% **<font color="ff6f61">수작업(Manual)</font>**으로 파라미터를 수정하는 방법입니다.<br/>
학계에서 논문을 출간할 수 있을 정도로 놀라운 정확도를 보여주는 하이퍼파라미터의 수치를 찾아내기 위해 쓰는 방법이죠.<br/>
이를 위해서 실험자의 경험이나 도메인 지식이 필요하기도 합니다.<br/>
~~*(물론 지도교수님들이 이 걸 직접 하시진 않습니다, 교수님의 시간은 소중하니까요...)*~~

2. **Grid Search**

    하지만 언제까지나 이렇게 하나하나 수작업으로만 시도해 볼 수는 없겠죠.<br/>
1번 방식을 자동화한 방법이 바로 **<font color="ff6f61">"Grid Search"</font>**입니다.<br/>
이 방법에서는 하이퍼파라미터마다 탐색할 지점을 정해주면 모든 지점에 해당하는 조합을 알아서 수행합니다.

    Grid Search는 학습을 실행한 뒤 한참 놀다오면 되는 매우 편한 방법이지만 **장점만 있는 것은 아닙니다.**<br/>
범위를 너무 많이 설정하면 '좀 놀다 오면 끝나는' 수준을 넘어 '수료하고 취직을 하고 나서도 끝나지 않을 수도' 있는데요.<br/>
만약 5개의 파라미터에 대해 각각 5개의 지점을 지정해주면 Grid Search는 총 $5^5=3,125$ 번의 모델 학습을 진행하게 됩니다.<br/>
여기에 5번의 교차 검증까지 진행한다면 모델은 $3,125 \times 5 = 15,625$ 번이나 학습을 수행합니다.<br/>
모델 한 번 학습에 10분만 걸린다고 쳐도 **3달 반**이 걸리는 무시무시한 작업입니다. 실제로 이런 일은 없어야겠죠?

    그렇기 때문에 Grid Search 로 너무 많은 하이퍼파라미터 조합을 찾으려고 하지 않는 것이 좋습니다.<br/>
1개, 혹은 최대 2개 정도의 파라미터 최적값을 찾는 용도로 적합합니다.<br/>
굳이 많은 하이퍼파라미터 조합을 시도할 필요는 없습니다.<br/>
모델 성능에 **보다 직접적인 영향을 주는 하이퍼파라미터가 따로 있기 때문**인데요.<br/>
이러한 파라미터만 제대로 튜닝해서 최적값을 찾은 후 나머지 하이퍼파라미터도 조정해나가면 못해도 90% 이상의 성능을 확보할 수 있습니다.<br/>
이런 식으로 하나씩 접근하다 보면 적어도 무한루프가 발생하는 위험은 줄일 수 있습니다.

3. **Random Search**

    **<font color="ff6f61">"Random Search"</font>** 는 무한 루프라는 Grid Search의 단점을 해결하기 위해 나온 방법입니다.<br/>
Random Search 는 지정된 범위 내에서 무작위로 모델을 돌려본 후 최고 성능의 모델을 반환합니다.<br/> 시도 횟수를 정해줄 수 있기 때문에 Grid Search 에 비해서 훨씬 적은 횟수로도 끝마칠 수 있겠죠?

    Grid Search 에서는 파라미터의 중요도가 모두 동등하다고 가정합니다.<br/>
하지만 위에서 알아본 것처럼 실제로 더 중요한 하이퍼파라미터가 있는데요.<br/>
Random Search 는 **상대적으로 중요한 하이퍼파라미터에 대해서는 탐색을 더 하고, 덜 중요한 하이퍼파라미터에 대해서는 실험을 덜 하도록** 합니다.

    Random Search 는 절대적으로 완벽한 하이퍼파라미터를 찾아주지는 않는다는 단점을 가지고 있는데요.<br/>
하지만 Grid Search와 비교했을 때, 학습에 걸리는 시간이 훨씬 더 적다는 점으로도 Random Search의 의의를 찾을 수 있습니다.

4. **Bayesian Methods**

    "Baby sitting" 이나 "Grid Search" 등의 방식에서는 탐색 결과를 보고, 결과 정보를 다시 새로운 탐색에 반영하면 성능을 더 높일 수 있었습니다.<br/> **<font color="ff6f61">베이지안 방식(Bayesian Method)</font> 은 이렇게 이전 탐색 결과 정보를 새로운 탐색에 활용하는 방법**입니다.<br/>
그렇기 때문에 베이지안 방법을 사용하면 하이퍼파라미터 탐색 효율을 높일 수 있습니다.<br/>
`bayes_opt` 나 `hyperopt`와 같은 패키지를 사용하면 베이지안 방식을 적용할 수 있습니다.

### 튜닝 가능한 파라미터에는 어떤 것이 있을까요?

신경망에서 탐색해 볼 수 있는 하이퍼파라미터의 종류는 다음과 같습니다.

- 배치 크기(**`batch_size`**)
- 에포크(**`epochs`**)
- 옵티마이저(**`optimizers`**)
- 학습률(**`learning rate`**)
- 활성화 함수(**`activation`**)
- Regularization(**`weight decay, Dropout`** 등)
- 은닉층(Hidden layer)의 노드(Node) 수

***실제로는 이보다 더 많은 하이퍼파라미터를 튜닝할 수 있습니다.<br/>


### GridSearch 를 사용한 최적의 배치 사이즈 탐색하기

당뇨병 데이터셋을 신경망에 적용해보고 배치 사이즈를 여러 개로 조정하면서 최적의 배치 사이즈를 찾아보겠습니다.



1. **필요한 패키지를 import 합니다.**

In [17]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

2. **재현성을 위해 랜덤시드를 고정합니다**

In [18]:
numpy.random.seed(42)

3. **데이터셋을 불러온 후에 Feature 와 Label로 분리합니다.**

In [19]:
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

In [20]:
X = dataset[:,0:8]
Y = dataset[:,8]

4. **모델을 제작합니다.**

    추후 **`KerasClassifier`** 로 Wrapping 하기 위하여 신경망 모델을 함수 형태로 정의합니다. 

In [21]:
def create_model():
    model = Sequential()
    model.add(Dense(100, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

4. **`KerasClassifier` 로 wrapping 하여줍니다.**

In [22]:
model = KerasClassifier(build_fn=create_model, verbose=0)

<ipython-input-22-7b0f05354f25>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


5. **하이퍼파라미터 탐색을 위한 탐색 범위를 설정한 후 `GridSearchCV` 를 지정하여 학습합니다.**

In [23]:
batch_size = [8, 16, 32, 64, 128]
param_grid = dict(batch_size=batch_size)

In [24]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

6. **최적의 결과를 낸 하이퍼파라미터와 각각의 결과를 출력해봅시다.**

In [25]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.6263135552406311 using {'batch_size': 8}
Means: 0.6263135552406311, Stdev: 0.04122325135357993 with: {'batch_size': 8}
Means: 0.5403870642185211, Stdev: 0.04648520364617009 with: {'batch_size': 16}
Means: 0.5704099893569946, Stdev: 0.10338463389652408 with: {'batch_size': 32}
Means: 0.47902555465698243, Stdev: 0.1175126271946634 with: {'batch_size': 64}
Means: 0.5209235310554504, Stdev: 0.13641581750465057 with: {'batch_size': 128}


## 라이브러리를 사용한 하이퍼파라미터 튜닝

### Keras Tuner 를 사용하여 하이퍼파라미터 탐색하기


**<font color="ff6f61">Keras Tuner</font>** 는 케라스 프레임워크에서 하이퍼파라미터를 튜닝하는 데 도움이 되는 라이브러리입니다.<br/>
Fashion MNIST 예제에 Keras Tuner를 적용하여 하이퍼파라미터 튜닝을 수행해보겠습니다.

1. **필요한 패키지를 import 합니다.**


In [26]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

import tensorflow as tf
import IPython

2. **Keras Tuner를 설치한 후 import 합니다.**

Keras Tuner는 Colab에 내장된 패키지가 아니기 때문에 따로 설치를 해준 후에 import 하여줍니다.

In [27]:
!pip install -U keras-tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.8 MB/s eta 0:00:00


3. **데이터셋을 불러온 후에 정규화(Normalizing) 해줍니다.**

    Fashion MNIST 데이터셋을 불러온 후에 이미지를 0-1 사이의 값으로 정규화합니다.

In [28]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [29]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

4. **Model을 제작합니다.**

모델을 제작하고 탐색할 하이퍼파라미터 범위와 지점을 정의합니다.<br/>

이 과정에서 Model builder 함수(**`model_builder`**)를 지정하는 과정이 필요합니다.<br/>
`model_builder` 라는 함수를 정의하고 해당 함수 내부에서 모델 설계와 하이퍼파라미터 튜닝까지 모두 수행해보겠습니다.

- **탐색할 하이퍼파라미터와 범위**
    - 은닉층의 노드 수 : 32 부터 512 까지 32개씩 증가시키며 탐색
    - 학습률(Learning rate) : 0.01, 0.001, 0.0001 의 3개 지점을 탐색


In [30]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(Flatten(input_shape=(28, 28)))
  
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(Dense(units = hp_units, activation = 'relu'))
  model.add(Dense(10, activation='softmax'))

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

5. **하이퍼파라미터 튜닝을 수행할 튜너(Tuner)를 지정합니다.**

Keras Tuner 에서는 **Random Search, Bayesian Optimization, Hyperband** 등의 최적화 방법을 수행할 수 있습니다.<br/>
아래에서는 **`Hyperband`** 를 통해서 튜닝을 수행해보도록 하겠습니다.

Hyperband 사용 시 Model builder function(**`model_builder`**), 훈련할 최대 epochs 수(**`max_epochs`**) 등을 지정해주어야 합니다.<br/>
Hyperband 는 리소스를 알아서 조절하고 조기 종료(Early-stopping) 기능을 사용하여 
높은 성능을 보이는 조합을 신속하게 통합한다는 장점을 가지고 있습니다.



In [31]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

6. **Callback 함수를 지정합니다.**

    하이퍼파라미터 탐색을 실행하기 전에 학습이 끝날 때마다 이전 출력이 지워지도록 콜백 함수를 정의해봅시다.

In [32]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

하이퍼파라미터 탐색을 수행합니다.
약 20분 소요


In [33]:
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
""")

Trial 30 Complete [00h 01m 23s]
val_accuracy: 0.8901000022888184

Best val_accuracy So Far: 0.8901000022888184
Total elapsed time: 00h 15m 36s

하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 192 입니다.
최적의 학습 속도는 0.001 입니다.



7. **최고 성능을 보이는 하이퍼파라미터 조합으로 다시 학습을 진행해봅시다.**

In [34]:
model = tuner.hypermodel.build(best_hps)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 192)               150720    
                                                                 
 dense_3 (Dense)             (None, 10)                1930      
                                                                 
Total params: 152,650
Trainable params: 152,650
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4844 - accuracy: 0.8295 - val_loss: 0.4253 - val_accuracy: 0.8429
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3650 - accuracy: 0.8669 - val_loss: 0.3759 - val_accuracy: 0.8648
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3272 - accuracy: 0.8799 - val_loss: 0.3967 - val_accuracy: 0.8548
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3040 - accuracy: 0.8881 - val_loss: 0.3548 - val_accuracy: 0.8705
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2863 - accuracy: 0.8945 - val_loss: 0.3556 - val_accuracy: 0.8762
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2713 - accuracy: 0.8983 - val_loss: 0.3472 - val_accuracy: 0.8729
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2586 - accuracy: 0.9039 - val_loss: 0.3279 - val_accuracy:

# 실험 기록 툴(wandb 등)에 대해 알아보기


- **실험 기록을 하는 이유는 무엇일까요?**

다양한 하이퍼파라미터를 변경해가면서 장기적으로 실험을 진행하다보면 점점 결과를 관리하기가 어려워집니다.<br/>
"어떤 파라미터 조합이 제일 좋았지?", "어제 했던 결과와 차이가 있었던가?" 와 같은 의문을 품게 되죠.<br/>
비록 노트북(`.ipynb`)이 어느정도 출력물을 기록하기는 있지만 모든 실험 결과를 관리하기엔 적절하지 않습니다.

**Comet.ml, Weights and Biases(`wandb`)** 등은 이러한 문제를 해결하기 위해 등장한 실험 기록 도구입니다.<br/>
이런 실험 기록 도구는 실험 결과를 실시간으로 기록하고 **코드와 결과값을 보관**해주며,<br/>
실험 결과를 원하는 기준대로 언제든지 **시각화하여 모델의 성능을 확인**할 수 있도록 도와줍니다.<br/>
매 Epoch이 끝날 때마다 데이터가 해당 툴에 보내지기 때문에 **모델이 수렴하고 있는지도 확인**할 수 있습니다.

이번 시간에는 Weights and Biases(`wandb`)를 활용하여 실험 기록을 수행해보도록 하겠습니다.

### Wandb 이용하기

- 설치 및 회원가입<br/>
먼저 다음 셀을 실행하기 전에 터미널에서 `wandb`에 로그인이 되있어야 합니다. 


구체적인 방법은 Weights and Biases의 [QuickStart](https://docs.wandb.com/quickstart)를 참고해주시면 되겠습니다.

In [ ]:
!pip install wandb

In [7]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
from wandb.keras import WandbCallback

In [9]:
# group, project 변수를 설정합니다. 반복되는 이름이 많기 때문에 변수로 설정하여 사용하면 편리합니다.
wandb_project = "review"
wandb_group = ""

In [15]:
!git clone http://github.com/wandb/tutorial

Cloning into 'tutorial'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 39 (delta 3), reused 6 (delta 2), pack-reused 31
Unpacking objects: 100% (39/39), 8.80 KiB | 819.00 KiB/s, done.


In [ ]:
!cd tutorial; pip install --upgrade -r requirements.txt;

In [17]:
import numpy
import pandas as pd
from tensorflow import keras
from tensorflow.python import keras
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV

In [18]:
!python -c "import keras; print(keras.__version__)"

2023-03-25 00:59:05.425547: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2.12.0


In [ ]:
wandb.init(project="wandb_prac")  ## 내가 만든 프로젝트 이름을 넣어주어야 합니다.
wandb.init(project="wandb_prac", entity=wand_group) 

# 데이터 및 하이퍼파라미터 설정 
X =  x_train
y =  y_train

inputs = X.shape[1]
wandb.config.epochs = 50
wandb.config.batch_size = 10

# 모델을 구축합니다
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# 모델을 컴파일 합니다
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# 모델을 학습합니다
model.fit(X, y, 
          validation_split=0.3, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

In [ ]:
wandb.init(project="wandb_prac")  ## 내가 만든 프로젝트 이름을 넣어주어야 합니다.

# 데이터 및 하이퍼파라미터 설정 
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

wandb.config.epochs = 10
wandb.config.batch_size = 64

# 모델을 구축합니다
model = Sequential() ## 과제시에는 이 모델을 Tre-trained model로 대체하면 됩니다. 
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()


In [ ]:
# 모델학습방식을 정의함
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습시키기
model.fit(train_images, train_labels, 
          validation_data=(test_images, test_labels),
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
          )

성능이 마음에 안든다면 추가로 학습을 더 시키는 방법도 있습니다. 

In [ ]:
wandb.config.epochs = 20
wandb.config.batch_size = 512

model.fit(train_images, train_labels, 
          validation_data=(test_images, test_labels),
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
          )

In [23]:
!ls wandb/

debug-internal.log  run-20230325_005648-2qodjmsb  run-20230325_010215-bn79os57
debug.log	    run-20230325_005910-uyfz6vm0
latest-run	    run-20230325_010119-7p39sf74
